In [1]:
import pandas as pd
import json
import getpass
import requests
import datetime
import os


In [2]:
WB_TOKEN = os.getenv('WB_TOKEN')
headers = {'Authorization': WB_TOKEN}

In [3]:
tarriffs = "https://common-api.wildberries.ru/api/v1/tariffs/box"
params = {
    "date": "2025-02-01"
}
response = requests.get(tarriffs,  headers=headers, params=params)


In [10]:
d = json.loads(response.text)
fbs_tarriffs = [{'boxDeliveryBase': row['boxDeliveryBase'], 'boxDeliveryLiter': row['boxDeliveryLiter'] } for row in d['response']['data']['warehouseList'] if row['warehouseName'] == 'Маркетплейс'][0]

boxDeliveryBase = fbs_tarriffs['boxDeliveryBase']
boxDeliveryLiter = fbs_tarriffs['boxDeliveryLiter']

print(f"base={boxDeliveryBase} liter={boxDeliveryLiter}")

base=40,25 liter=9,78


In [12]:
def get_cards_list(api_key, updated_at=None, nm_id=None, limit=100):
    headers = {
        "Authorization": api_key,
        "Content-Type": "application/json"
    }
    
    cards_list_url = "https://content-api.wildberries.ru/content/v2/get/cards/list"
    
    # Базовый запрос
    body = {
        "settings": {
            "cursor": {
                "limit": limit
            },
            "filter": {
                "withPhoto": 1,
                "objectIDs": [3979]
            }
        }
    }
    
    # Добавляем параметры, если они переданы
    if updated_at:
        body["settings"]["cursor"]["updatedAt"] = updated_at
    if nm_id:
        body["settings"]["cursor"]["nmID"] = nm_id
    
    try:
        response = requests.post(cards_list_url, headers=headers, json=body)
        response.raise_for_status()  # Выбросит исключение, если статус ответа не 200
        return response.json()
    except requests.RequestException as e:
        print(f"Ошибка при запросе списка карточек: {e}")
        return None


In [14]:
all_cards = []
updated_at = None
nm_id = None

while True:
    response = get_cards_list(WB_TOKEN, updated_at=updated_at, nm_id=nm_id)

    if not response or "cards" not in response:
        print("Запрос завершён или нет данных.")
        break

    cards = response["cards"]
    if not cards:
        print("Больше нет карточек.")
        break

    all_cards.extend(cards)

    # Обновляем параметры для следующего запроса
    cursor = response.get("cursor", {})
    updated_at = cursor.get("updatedAt")
    nm_id = cursor.get("nmID")

    # Если нет курсора, значит, это последняя страница
    if not updated_at or not nm_id:
        break

    print(f"Загружено {len(all_cards)} карточек, продолжаем...")



Загружено 100 карточек, продолжаем...
Загружено 200 карточек, продолжаем...
Загружено 273 карточек, продолжаем...
Больше нет карточек.


In [15]:
all_cards

[{'nmID': 322405153,
  'imtID': 303872160,
  'nmUUID': '0194b77b-e657-70e2-a436-2b56918d428c',
  'subjectID': 3979,
  'subjectName': 'Коробки картонные',
  'vendorCode': 'box_9603049_10',
  'brand': '',
  'title': 'Коробка крафт кондитерская, беленая, 140*140*60 мм 10 шт.',
  'needKiz': False,
  'photos': [{'big': 'https://basket-19.wbbasket.ru/vol3224/part322405/322405153/images/big/1.webp',
    'c246x328': 'https://basket-19.wbbasket.ru/vol3224/part322405/322405153/images/c246x328/1.webp',
    'c516x688': 'https://basket-19.wbbasket.ru/vol3224/part322405/322405153/images/c516x688/1.webp',
    'hq': 'https://basket-19.wbbasket.ru/vol3224/part322405/322405153/images/hq/1.webp',
    'square': 'https://basket-19.wbbasket.ru/vol3224/part322405/322405153/images/square/1.webp',
    'tm': 'https://basket-19.wbbasket.ru/vol3224/part322405/322405153/images/tm/1.webp'}],
  'dimensions': {'width': 56, 'height': 3, 'length': 40, 'isValid': True},
  'characteristics': [{'id': 90673, 'name': 'Ширин